In [1]:
# !pip install python-binance
import os
from interact_with_binance import fetch_ohlcv
from market_imbalance import MarketCandle
from binance.client import Client


api_key = os.environ.get('BINANCE_API_KEY')
api_secret = os.environ.get('BINANCE_API_SECRET')
client = Client(api_key, api_secret)

# Récupère les données de marché en *temps différé* sur Binance
interval_value = Client.KLINE_INTERVAL_1MINUTE
historical_candles_data = fetch_ohlcv(client, symbol = "BTCUSDT", interval=interval_value, from_date= "1 avril 2024") #"1 Jan, 2015"

# Stocke les informations de chaque bougie dans un objet afin de pouvoir les retravailler plus facilement par la suite
historical_candles_object = [MarketCandle(*candle) for candle in historical_candles_data]

ModuleNotFoundError: No module named 'interact_with_binance'

In [ ]:
from market_imbalance import imbalances_after_fall, fullfilled_imbalances_after_fall, imbalances_after_rise, fullfilled_imbalances_after_rise

# Percentage of gap filled after fall
percentage_filled_after_fall = (len(fullfilled_imbalances_after_fall) / len(imbalances_after_fall)) * 100
print(f'% of filled gap after fall = {percentage_filled_after_fall} with {len(fullfilled_imbalances_after_fall)} fullfilled imbalances on {len(imbalances_after_fall)} in total')

# Percentage of gap filled after rise
percentage_filled_after_rise = (len(fullfilled_imbalances_after_rise) / len(imbalances_after_rise)) * 100
print(f'% of filled gap after rise = {percentage_filled_after_rise} with {len(fullfilled_imbalances_after_rise)} fullfilled imbalances on {len(imbalances_after_rise)} in total')